<a href="https://colab.research.google.com/github/JunSeokCheon/Natural-Language-based-Climate-Technology-Classification/blob/master/LGBM(Okt).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install konlpy

In [ ]:
pip install lightgbm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
import tqdm

from konlpy.tag import Okt

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score,f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
cd /content/drive/MyDrive/daicon

In [ ]:
ls

In [ ]:
train=pd.read_csv('preprocessing_train.csv')
test=pd.read_csv('preprocessing_test.csv')
sample_submission=pd.read_csv('sample_submission.csv')

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train.head(20)

In [ ]:
test.head(2)

In [ ]:
sample_submission.head(6)

In [ ]:
#데이터 구조 파악
print(train.shape)
print(test.shape)
print(sample_submission.shape)

In [ ]:
#심각한 불균형 데이터임을 알 수 있습니다.
train.label.value_counts(sort=False)/len(train)

In [ ]:
# train = train[['과제명', '요약문_한글키워드', '요약문_영문키워드','label']]
# test = test[['과제명', '요약문_한글키워드', '요약문_영문키워드']]
# train[['과제명', '요약문_한글키워드', '요약문_영문키워드', 'label']].fillna('', inplace=True)
# test[['과제명', '요약문_한글키워드', '요약문_영문키워드']].fillna('', inplace=True)

In [ ]:
train = train[['요약문_영문키워드', 'label']]
test = test[['요약문_영문키워드']]
train[['요약문_영문키워드', 'label']].fillna('', inplace=True)
test[['요약문_영문키워드']].fillna('', inplace=True)

In [ ]:
train.head(2)

In [ ]:
train['data'] = train['요약문_영문키워드']
test['data'] = test['요약문_영문키워드'] 

In [ ]:
train.head(2)

In [ ]:
#1. re.sub 한글 및 공백을 제외한 문자 제거
#2. okt 객체를 활용해 형태소 단위로 나눔
#3. remove_stopwords로 불용어 제거 
def preprocessing(text, okt, remove_stopwords=False):
    # text=re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ]","", text)
    word_text=okt.morphs(text, stem=True)
    if remove_stopwords:
        word_review=[token for token in word_text if not len(token)==1]
    return word_review

In [ ]:
# stop_words=['은','는','이','가', '하','아','것','들','의','있','되','수','보','주','등','한']
okt=Okt()
clean_train_text=[]
clean_test_text=[]

In [ ]:
#시간이 많이 걸립니다.
for text in tqdm.tqdm(train['data']):
    try:
        clean_train_text.append(preprocessing(text, okt, remove_stopwords=True))
    except:
        clean_train_text.append([])

In [ ]:
clean_train_text[5]

In [ ]:
for text in tqdm.tqdm(test['data']):
    if type(text) == str:
        clean_test_text.append(preprocessing(text, okt, remove_stopwords=True))
    else:
        clean_test_text.append([])

In [ ]:
len(clean_train_text)

In [ ]:
#추후 재사용 가능하도록 npy로 전환
DATA_IN_PATH='./data_in/'
TRAIN_INPUT_DATA = 'train_input_ENkey.npy'
TEST_INPUT_DATA = 'test_input_ENkey.npy'

import os
if not os.path.exists(DATA_IN_PATH):
    os.makedirs(DATA_IN_PATH)
    
np.save(open(DATA_IN_PATH+TRAIN_INPUT_DATA, 'wb'), clean_train_text)
np.save(open(DATA_IN_PATH+TEST_INPUT_DATA, 'wb'), clean_test_text)

In [ ]:
len(clean_test_text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#tokenizer 인자에는 list를 받아서 그대로 내보내는 함수를 넣어줍니다. 또한 소문자화를 하지 않도록 설정해야 에러가 나지 않습니다.
vectorizer = CountVectorizer(tokenizer = lambda x: x, lowercase=False)
train_features=vectorizer.fit_transform(clean_train_text)
test_features=vectorizer.transform(clean_test_text)
#test데이터에 fit_transform을 할 경우 data leakage에 해당합니다
#https://deepinsight.tistory.com/165

In [ ]:
train_features

In [ ]:
test_features

In [ ]:
import lightgbm as lgb
from sklearn import metrics as ms

In [ ]:
pip install optuna

In [ ]:
import optuna

In [ ]:
def objective(trial):
    TEST_SIZE=0.2
    RANDOM_SEED=42

    train_x, eval_x, train_y, eval_y=train_test_split(train_features, train['label'], test_size=TEST_SIZE, random_state=RANDOM_SEED)

    train_x = train_x.astype('float32')
    eval_x = eval_x.astype('float32')
    train_y = train_y.astype('float32')
    eval_y = eval_y.astype('float32')
    
    train_ds = lgb.Dataset(train_x, label = train_y)
    
 
    param = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'num_classes': '46',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
 
    gbm = lgb.train(param, train_ds)
    y_pred = gbm.predict(eval_x).argmax(axis=1)
    accuracy = ms.accuracy_score(y_pred,eval_y)
    return accuracy
 
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)
 
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
#훈련 데이터 셋과 검증 데이터 셋으로 분리
TEST_SIZE=0.2
RANDOM_SEED=42

train_x, eval_x, train_y, eval_y=train_test_split(train_features, train['label'], test_size=TEST_SIZE, random_state=RANDOM_SEED)

In [ ]:
len(eval_y)

In [ ]:
train_x = train_x.astype('float32')
eval_x = eval_x.astype('float32')
train_y = train_y.astype('float32')
eval_y = eval_y.astype('float32')

In [ ]:
import lightgbm as lgb

train_ds = lgb.Dataset(train_x, label = train_y)
test_ds = lgb.Dataset(eval_x, label = eval_y)

In [ ]:
params = {
    'learning_rate' : 0.01,
    'max_depth' : 16,
    'boosting' : 'gbdt',
    'num_classes': '46',
    'objective' : 'multiclass',
    'is_training_metric' : True,
    'lambda_l1': 3.6389990001909315e-05, 'lambda_l2': 0.10815227244766439, 'num_leaves': 199, 'feature_fraction': 0.8192426309847745, 'bagging_fraction': 0.9050477494813629, 'bagging_freq': 1, 'min_child_samples': 5
}

In [ ]:
import tensorflow as tf

In [ ]:
with tf.device('/device:GPU:0'):
  model = lgb.train(params, train_ds, 1000, test_ds, verbose_eval=100, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's multi_logloss: 0.691473
[200]	valid_0's multi_logloss: 0.617426
[300]	valid_0's multi_logloss: 0.57758
[400]	valid_0's multi_logloss: 0.552808
[500]	valid_0's multi_logloss: 0.535492
[600]	valid_0's multi_logloss: 0.522389
[700]	valid_0's multi_logloss: 0.512284
[800]	valid_0's multi_logloss: 0.503911
[900]	valid_0's multi_logloss: 0.496876
[1000]	valid_0's multi_logloss: 0.490848
Did not meet early stopping. Best iteration is:
[1000]	valid_0's multi_logloss: 0.490848


In [ ]:
y_pred = model.predict(eval_x).argmax(axis=1)

In [ ]:
y_pred

In [ ]:
from sklearn import metrics as ms
ms.accuracy_score(y_pred,eval_y)

In [ ]:
len(y_pred)

In [ ]:
len(eval_y)

In [ ]:
sample_submission['label'] = model.predict(test_features.astype("float32")).argmax(axis=1)

In [ ]:
sample_submission

,index,label
0,174304,0
1,174305,0
2,174306,0
3,174307,0
4,174308,0
...,...,...
43571,217875,0
43572,217876,0
43573,217877,0
43574,217878,0


In [ ]:
sample_submission.to_csv('LGBM_baseline_ENkey.csv', index=False)